<h1><center>Laboratorio 8: ¿Superhéroe o Villano? 🦸</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Matías Rojas y Mauricio Araneda
- Auxiliar: Ignacio Meza D.
- Ayudante: Rodrigo Guerra

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Cristóbal Alcázar
- Nombre de alumno 2: Gianina Salomó


### **Link de repositorio de GitHub:** `https://github.com/alcazar90/sci-prog-lab`

## Temas a tratar

- Codificación de texto usando Bag of Words.
- Búsqueda del modelo óptimo de clasificación usando `GridSearch`
- Uso de pipelines.

## Reglas:

- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Obtener caracteristicas a partir de texto usando `CountVectorizer`.
- Fijar un pipeline con un modelo base que luego se irá optimizando.
- Comprender como realizar una búsqueda de grilla sobre un conjunto de clasificadores e hiperparámetros usando `GridSearch`.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

#Importamos librerias utiles 😸

In [ ]:
# Librería Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn import set_config

from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import FunctionTransformer

# Clasifación
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

# Búsqueda de grilla
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

# Librería para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Proyecciones en baja dimensionalidad: UMAP
!pip install umap-learn

# Librería para NLP
!pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize  
from nltk.stem import PorterStemmer
nltk.download(['stopwords', 'punkt'])

# Serialización
import pickle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.3 MB 31.3 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 88 kB 5.8 MB/s 
     |████████████████████████████████| 1.1 MB 34.9 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82830 sha256=f11274c5d23ccb75144b14979868cf931ce28bce28ebdc903e6df27ed1172de8
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.8-py3-none-any.whl size=55509 sha256=23ce2296a3a373915e3611aa020334402c06bd0cc927caefe80b3ac94cb435d1
  Stored in directory: /root/.cache/pip/wheels/19/bc

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# 1. ¿Quien es Bat Cow?

<p align="center">
  <img src="https://i.imgur.com/D9f1RHy.jpg" width="350">
</p>

En vez de estar desarrollando las evaluaciones correspondientes a su curso, su profesor de catedra y su auxiliar discuten acerca la alineación (héroe o villano) del personaje de ficción Bat-Cow. 

El cuerpo docente, no logra ponerse de acuerdo si el personaje es bueno, neutral o malo: el auxiliar plantea que Bat-cow posee una siniestra mirada, intrigante pero común característica de los personajes malvados. 
Por otra parte, extendiendo las ideas de Rousseau, el profesor plantea que tal como los humanos no nacen malos, no existe motivo por el cual una vaca con superpoderes deba serlo.

Sin embargo, ambos concuerdan que es difícil estimar la alineación solo usando los atributos físicos, por lo que creen el análisis debe ser complementado aún más antes de comunicarle los resultados a su estudiantado. Buscando más información, ambos sujetos se percatan de la existencia de un excelente antecedente para estimar la alineación: la historia personal de cada superhéroe o villano.

Es por esto le solicitan que construya y optimice un clasificador basado en texto el cual analice la alineación de cada personaje basado en su historia personal.

Para este laboratorio deben trabajar con los datos `df_comics.csv` y `comics_no_label.csv` subidos a u-cursos. El primero es un conjunto de datos que les servirá para entrenar un modelo de clasificación, mientras que el segundo es un dataset con personajes de ficción no etiquetados a predecir (sí, aquí está la misteriosa Batcow).

Para comenzar cargue los dataset señalados y visualice a través de un head los atributos que poseen cada uno de los dataset.


In [ ]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


In [ ]:
df_comics = pd.read_csv('https://raw.githubusercontent.com/alcazar90/sci-prog-lab/main/data/df_comics.csv', index_col=0)
df_comics_no_label = pd.read_csv('https://raw.githubusercontent.com/alcazar90/sci-prog-lab/main/data/comics_no_label.csv')
df_comics = df_comics.dropna(subset=['history_text']) # eliminar ejemplos sin historia

In [ ]:
# queda a labor de su equipo hacer el análisis exploratorio
df_comics.head()

,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,durability_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,3-D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...",NaN,85,30,60,60,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,A-Bomb,Richard Milhouse Jones,Richard Milhouse Jones,20,"Richard ""Rick"" Jones was orphaned at a young ...","On rare occasions, and through unusual circu...",80,100,80,100,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
3,Aa,Aa,NaN,12,Aa is one of the more passive members of the P...,NaN,80,50,55,45,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aaron Cash,Aaron Cash,Aaron Cash,5,Aaron Cash is the head of security at Arkham A...,NaN,80,10,25,40,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Aayla Secura,Aayla Secura,NaN,8,ayla Secura was a Rutian Twi'lek Jedi Knight (...,NaN,90,40,45,55,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


### Exploración de frecuencias del vector objetivo

In [ ]:
df_comics["alignment"].value_counts()

Good       743
Bad        429
Neutral    113
Name: alignment, dtype: int64

Las clases están desbalanceadas, lo cual puede significar un problema al momento de realizar predicciones certeras, sobre todo de la clase Neutral.

## 1.1 Obtención de Features y Bag of Words

<p align="center">
  <img src="https://media0.giphy.com/media/eIUpSyzwGp0YhAMTKr/200.gif" width="300">
</p>

Primero que todo, deben obtener un vector de características del atributo `history_text`, utilizando `Bag of Words`. En este atributo se presenta una breve descripción de la historia de cada uno de los personajes de ficción presentes en el dataset. 

Pero... antes de empezar, ¿Que es `Bag of Words`?...

`Bag of Words` es un modelo de conteo utilizado en Procesamiento de Lenguaje Natural (NLP) que tiene como objetivo generar una representación vectorial (vector de características en nuestro cas) para cada documento a través del conteo de las palabras que contienen. 

La siguiente figura muestra un ejemplo de `Bag of Words` en acción:

<p align="center">
  <img src="https://user.oc-static.com/upload/2020/10/23/16034397439042_surfin%20bird%20bow.png" width="500">
</p>

Como pueden ver, el modelo de `Bag of Words` no resulta tan complicado, ¿pero cómo lo aplicamos en python?. 

Como podrán darse cuenta del ejemplo anterior, para facilitar el conteo será necesario transformar cada uno de los documentos en vectores, donde cada una de las posiciones posee un carácter. Este proceso es conocido como **tokenización** y lo podemos realizar de la siguiente forma:

In [ ]:
docs = ['The teacher rocks like a good rock & roll',
        'the rock is the best actor in the world']


docs_tokenizados = [word_tokenize(doc)  for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world']]

Podemos mejorar un poco más el proceso de tokenización agregando 

- **Stemming:**  Definimos Stemming como un algoritmo basado en reglas que transforma las palabras a una forma general. Un ejemplo de stemming, es el siguiente:
<p align="center">
  <img src="https://devopedia.org/images/article/218/8583.1569386710.png" width="300">
</p>
- **Eliminación de Stopwords:** Eliminación de palabras muy frecuentes que entorpecen la clasificación (por ejemplo, el, la los, la, etc...)


In [ ]:
# Definimos algunas stopword que queremos que sean eliminadas
import nltk
#nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words = stopwords.words('english')

# Definimos un tokenizador con Stemming
class StemmerTokenizer:
    def __init__(self):
        self.ps = PorterStemmer()
        
    def __call__(self, doc):
        doc_tok = word_tokenize(doc)
        doc_tok = [t for t in doc_tok if t not in stop_words]
        return [self.ps.stem(t) for t in doc_tok]

# Inicializamos tokenizador
tokenizador = StemmerTokenizer()

# Creamos algunos documentos
docs = ['The teacher rocks like a good rock & roll',
        'the rock is the best actor in the world',
        'New York is a beautiful city']

# Obtenemos el token del primer documento
[tokenizador(doc) for doc in docs]

[['the', 'teacher', 'rock', 'like', 'good', 'rock', '&', 'roll'],
 ['rock', 'best', 'actor', 'world'],
 ['new', 'york', 'beauti', 'citi']]

In [ ]:
# Comparación con el caso anterior
docs_tokenizados = [word_tokenize(doc) for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['New', 'York', 'is', 'a', 'beautiful', 'city']]

#### Al Estilo Scikit

Scikit implementa `bag of words` a través de la clase `CountVectorizer()` la cual contiene muchas opciones para mejorar la tokenización.

In [ ]:
bow = CountVectorizer(tokenizer= StemmerTokenizer())
# CounterVectorizer() funciona con la típica interfaz de sklearn fit/fit_transform:
df = bow.fit_transform(docs)

# .get_feature_names_out() entrega el corpus de palabras encontradas por el tonekizador 
# en los documentos
pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

,&,actor,beauti,best,citi,good,like,new,rock,roll,teacher,world,york
0,1,0,0,0,0,1,1,0,2,1,1,0,0
1,0,1,0,1,0,0,0,0,1,0,0,1,0
2,0,0,1,0,1,0,0,1,0,0,0,0,1


In [ ]:
# La transformación originalmente entrega una "sparse matrix" (+ eficiente en memoria)
# pero es posible transformar en un dataframe como se hizo arriba
df

<3x13 sparse matrix of type '<class 'numpy.int64'>'
	with 14 stored elements in Compressed Sparse Row format>

Una de las cosas más interesantes que provee son el use de n-gramas, los cuales, en palabras simples, son conjuntos de n-palabras que se concatenan entre si y que se consideran como tokens separados. 

Pensemos en `Nueva York`. Cuando se tokeniza Nueva York, se generan dos tokens independientes que a simple vista no tienen relación: `Nueva` `York`.
Al usar n-gramas (en un rango min=1,max=2) , generamos tanto `Nueva` y `York` como también `Nueva York` como un token independiente.

In [ ]:
bow = CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,2))
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

,&,& roll,actor,actor world,beauti,beauti citi,best,best actor,citi,good,...,rock,rock &,rock best,rock like,roll,teacher,teacher rock,world,york,york beauti
0,1,1,0,0,0,0,0,0,0,1,...,2,1,0,1,1,1,1,0,0,0
1,0,0,1,1,0,0,1,1,0,0,...,1,0,1,0,0,0,0,1,0,0
2,0,0,0,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1


De los resultados, podemos ver que generamos vectores de conteo para cada una de las palabras que conforman el corpus.  Un punto extra que se agrega en esta obtención de frecuencias son los bigramas, que básicamente son el conjunto de palabras de tamaño de aparecen juntas en el texto.

## Codificando los Super{heroes, villanos}  [0.5 Puntos]

<p align="center">
  <img src="https://c.tenor.com/LkQzw7k5DV4AAAAd/anime-hacking.gif" width="300">
</p>

Conociendo ahora que es el proceso de `bag of words`, aplique este modelo de obtención de caracteristicas de la siguiente forma en un pipeline:

- Utilice el tokenizador entregado.
- Obtenga caracteristicas de los unigramas y bigramas del texto (tal como el ejemplo).

```python
bog = CountVectorizer(tokenizer= StemmerTokenizer(),`
                      ngram_range=(1,2) # Este punto es opcional y es para generar bigramas
                      )
```

Finalmente, aplique `MinMaxScaler()` sobre `atributos_de_interes` y concatene el valor obtenido con el matriz de caracteristicas obtenidas con bag of words.

```python
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']
```

No es necesario que obtenga un dataframe en concreto con las características solicitadas. Se le recomienda generar un `ColumnTransformer()` para aplicar las transformaciones solicitadas en un pipeline.

**To-Do:**
- [X] Obtener a traves de Bag of Words (`CountVectorizer`) caracteristicas del resumen de historia de cada personaje.
- [X] Aplicar `MinMaxScaler` sobre los atributos de interes.

**Respuesta:**

In [ ]:
# ¿Hay NAs en las columnas?
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score',
                        'durability_score', 'power_score', 'combat_score']
atributo_a_tokenizar = 'history_text'

df_comics[atributos_de_interes].isna().sum(axis=0)

intelligence_score    0
strength_score        0
speed_score           0
durability_score      0
power_score           0
combat_score          0
dtype: int64

Se crea el _pipeline_ `preprocessing` que concatena dos transformaciones
sobre columnas de interes:

* `num_pipe`: _pipeline_ encargado de aplicar la transformación `MinMaxScaler` 
sobre los atributos de interes númericos
* `bag_of_words`: _pipeline_ encargado de aplicar las transformaciones para
obtener el _bag of words_ del atributo de texto de interes


Notas que a transformación sobre el atributo de texto `history_text` genera
una matriz a partir de un vector, por lo que se debe considerar este
cambio de estructura en el Pipeline...

In [ ]:
# Pipeline con la transformación encargada de escalar
num_pipe = Pipeline(
    steps=[
        ("scaler", MinMaxScaler()),
    ]
)

# Pipeline con la transformación encargada de tokenizar
bag_of_words = Pipeline(
    steps=[
        ("tokenizer", CountVectorizer(tokenizer=StemmerTokenizer(), ngram_range=(1,2))),
    ]
)

# Las transformaciones se aplican en tuplas que asocian Pipelines a grupos de
# columnas
preprocessing = Pipeline([
    ("column_transform", ColumnTransformer(transformers=[
        ("num", num_pipe, atributos_de_interes),
        ("words", bag_of_words, atributo_a_tokenizar),
    ]
  ))
])

Verificamos si las transformaciones se están aplicando correctamente usando
solo 100 observaciones:

In [ ]:
df = preprocessing.fit_transform(df_comics.head(n=100))
pd.DataFrame(df.toarray())  

,0,1,2,3,4,5,6,7,8,9,...,36106,36107,36108,36109,36110,36111,36112,36113,36114,36115
0,0.85,0.30,0.60,0.60,0.40,0.70,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.80,1.00,0.80,1.00,1.00,0.80,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.80,0.50,0.55,0.45,1.00,0.55,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.80,0.10,0.25,0.40,0.30,0.50,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.90,0.40,0.45,0.55,0.55,0.85,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.65,0.55,0.25,0.55,0.20,0.55,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,0.90,0.15,0.25,0.50,0.50,1.00,0.0,0.0,0.0,0.0,...,1.0,4.0,0.0,0.0,2.0,0.0,1.0,1.0,0.0,0.0
97,1.00,0.30,0.65,0.60,1.00,0.90,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,1.00,0.30,0.50,0.50,0.30,1.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 1.2 Diseño de Baseline y  Primer Entrenamiento  [1 Puntos]


<p align="center">
  <img src="https://pa1.narvii.com/6374/9eaec1b7bf9157334151452a669516f9a78b954c_hq.gif" width="300">
</p>



Genere un Pipeline con las caracteristicas solicitadas en la sección 1.1, un selector de mejores features `SelectPercentile` con métrica `f_classif` y percentile=90 y un clasificador `MultinomialNB()` por defecto.

Luego, separe el conjunto de datos en un conjunto de entrenamiento y prueba, donde las etiquetas estará dado por el atributo `alignment`. 

Entrene el modelo y reporte el desempeño con un `classification_report`. ¿ Nos recomendaría predecir la alineación de BatCow con este clasificador?.

Finalmente, compare el modelo entrenado con un modelo Dummy estratificado y responda: ¿El clasificador entrenado es mejor que el dummy que entrega respuestas al azar?

**To-do:**
- [X] Realizar un pipeline con las caracteristicas solicitadas en 1.1, ejecutar holdout y aplicar un clasificador `MultinomialNB()`.
- [x] Entrenar el pipeline, calcular el `classification_report` asociado y comentar los resultados.
- [X] Entrenar un `DummyClassifier` con estrategia `statified`, calcular el `classification_report` asociado y comentar que implican los scores obtenidos en comparación con los resultados del baseline.

**Respuesta:**

In [ ]:
# Realizar un pipeline con las caracteristicas solicitadas en 1.1,
# ejecutar holdout y aplicar un clasificador MultinomialNB().

selection_pipeline = Pipeline(
    steps=[
        ("Preprocessing", preprocessing),
        # Conservamos el 90% mejor según la métrica seleccionada
        ("Selection", SelectPercentile(f_classif, percentile=90)),
        ("MNB", MultinomialNB()),
    ]
)

X_train, X_test, y_train, y_test = train_test_split(
    df_comics[atributos_de_interes + [atributo_a_tokenizar]],
    df_comics["alignment"],
    test_size=0.33,
    random_state=42,
    shuffle=True,
    stratify=df_comics["alignment"],
)

In [ ]:
# Entrenar el pipeline,
# calcular el classification_report asociado y comentar los resultados.
selection_pipeline.fit(X_train, y_train)

print(classification_report(y_test, selection_pipeline.predict(X_test)))

              precision    recall  f1-score   support

         Bad       0.76      0.13      0.23       142
        Good       0.61      0.99      0.75       246
     Neutral       0.00      0.00      0.00        37

    accuracy                           0.62       425
   macro avg       0.46      0.37      0.33       425
weighted avg       0.61      0.62      0.51       425



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Comentarios**

- El modelo entrenado en general tiene un buen desempeño para predecir los registros clasificados como "Good", sobre todo considerando `recall`, lo que indica pocos falsos negativos.
- El modelo entrenado tiene muchos falsos negativos en las predicciones de "Bad", lo que se traduce en un bajo `recall` y por consecuencia un bajo `f1` para esta clase (pese a tener buena precisión).
- El modelo entrenado no fue capaz de predecir ningún registro como "Neutral", pese a que existían 37 registros de esta categoría en los datos de prueba. Esto puede deberse en parte a que se tiene muy pocos datos de esta clase, pero también es posible que el modelo escogido y/o pre procesamiento de texto y/o selección de características no haya sido el más adecuado.

In [ ]:
# Entrenar un DummyClassifier con estrategia statified,
# calcular el classification_report asociado y comentar que implican los scores
# obtenidosen comparación con los resultados del baseline.
dummy_pipe = Pipeline(
    steps=[
        ("Preprocessing", preprocessing),
        ("dummy", DummyClassifier(strategy="stratified", random_state=42)),
    ]
)

dummy_pipe.fit(X_train, y_train)

print(classification_report(y_test, dummy_pipe.predict(X_test)))

              precision    recall  f1-score   support

         Bad       0.31      0.32      0.31       142
        Good       0.60      0.59      0.59       246
     Neutral       0.10      0.11      0.10        37

    accuracy                           0.45       425
   macro avg       0.34      0.34      0.34       425
weighted avg       0.46      0.45      0.46       425



**Comentarios**

- El "modelo dummy" tiene mejor desempeño en comparación al modelo anterior tanto para la clase "Bad" como "Neutral".
- Si bien en cuanto `accuracy` su desempeño aparentemente es peor, ya qie tiene 0.48 versus 0.62 del modelo `MultinomialNB`, se debe considerar que esta métrica no es un buen indicador frente a clases desbalanceadas. El modelo `MultinomialNB` logra un mejor accuracy ya que tiene muchas predicciones correctas de la clase mayoritaria, pero en las clases de menor frecuencia tiene pocas predicciones correctas.
- Por tanto, considerando "predicciones correctas totales", el modelo `MultinomialNB` logra un mejor desempeño que el clasificador aleatorio, pero no así al considerar en específico las clases de menor frecuencia que son "Bad" y "Neutral".

```
Escriba aquí su justificación
```
**¿ Nos recomendaría predecir la alineación de BatCow con este clasificador?**

No, ya que el modelo `MultinomialNB` no logra predecir correctamente la alineación "Neutral", y tiene mucho error en la alineación "Bad". El ideal es que el modelo sea capaz de predecir mejor que el azar las tres alineaciones solicitadas.

**El clasificador entrenado es mejor que el dummy que entrega respuestas al azar?**

No, porque según lo argumentado previamente, el clasificador `MultinomialNB` si bien logra mayor cantidad de clasificaciones totales correctas que el clasificador aleatorio, no es capaz de predecir las alineaciones "Neutral", y tiene mucho error al predecir las alineaciones "Bad".

## 1.3 Busqueda del Mejor Modelo con Grid Search [4 Puntos]

<p align="center">
  <img src="https://media1.tenor.com/images/70fdfeea52a8e2e4505498c230a0d2f9/tenor.gif?itemid=5134219" width="250">
</p>

No conformes con el rendimiento obtenido en la sección 1.2, el cuerpo docente les pide que realicen un **`HalvingGridSearchCV`** con diferentes parámetros para mejorar el rendimiento de la clasificación. Para esto, se le solicita que defina:

- Tres clasificadores distintos en donde varie sus parámetros. Considere usar modelos clásicos como también los basados en ensamblaje.
- Modificar `n-gram` range del `CountVectorizer` probando `(1,1), (1,2) y (1,3)`. Examinar también los otros parámetros de CountVectorizer como por ejemplo `max_df`, `min_df`, etc... ([Documentación aquí](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html))
- Seleccionar las columnas que contribuyen con la mayor información para la clasificación con `SelectPercentile` en los percentiles `[20, 40, 60, 80]` (puede usar la métrica que usted quiera).
- Reporte la mejor combinación encontrada y justifique por qué cree que es la mejor según el clasificador usado, la cantidad de columnas seleccionadas y los parámetros de CountVectorizer seleccionados por GridSearch.

A continuación, un ejemplo de parametros para GridSearch para una búsqueda de 3 clasificadores distintos:

```python
params = [
       # clasificador 1 + hiperparámetros
       {'clf': classificator1(),
        'clf__penalty': ['ovr'],
       # clasificador 1 + hiperparámetros    
       {'clf': classificator2(),
        'clf__n_estimators': [200]},
       # clasificador 1 + hiperparámetros
       {'clf': classificator3(),
        ...
       }
       ]
```

**Nota 1**: Puede ver los parámetros modificables aplicando el método get_params() sobre su pipeline. Ver la clase de GridSearch para mayor información sobre la sintáxis de las grillas.

**Nota 2**: Recuerde inicializar los clasificadores con un random state definido.

**Nota 3**: Puede usar en `HalvingGridSearchCV` el parámetro `verbose=10` para ver que GridSearch le indique el estado de su ejecución.

**Nota 3:** El GridSearch puede tomar tiempos de búsqueda exorbitantes, por lo que se le recomienda no agrandar mucho el espacio de búsqueda, dejar corriendo el código y tomarse un tecito.

**Respuesta:**

Antes de comenzar con la grilla de parámetros y la búsqueda del mejor
modelo, realizaremos una modificación en el _pipeline_ `preprocessing`,  para que la transformación que se aplica sobre la columna
de texto `history_text`, en el paso `tokenizer`, entregué un `np.array` y no un objeto tipo _sparse matrix_. Esto evitará que se produzcan errores con los
modelos candidatos que utilizaremos y esperan recibir un _input_ con un
formato determinado.

Agregaremos la función `toarray()` que asegura que el _input_ siempre sea
un `np.array`:

```python
def toarray(x): return x.toarray()
```

y luego utilizamos el _wrapper_ `FunctionTransformer`, el cual adapta
nuestra transformación a un objeto que el _pipeline_ de sklearn pueda
manipular de manera coherente con el resto de la interfaz.

In [ ]:
from sklearn.preprocessing import FunctionTransformer
set_config(display='diagram')

def toarray(x): return x.toarray()

bag_of_words = Pipeline(
    steps=[
        ("tokenizer", CountVectorizer(tokenizer=StemmerTokenizer(), ngram_range=(1,2))),
        ("toarray", FunctionTransformer(toarray, accept_sparse=True))
    ]
)

preprocessing = Pipeline([
    ("column_transform", ColumnTransformer(transformers=[
        ("num", num_pipe, atributos_de_interes),
        ("words", bag_of_words, atributo_a_tokenizar),
    ]
  ))
])

preprocessing

Pipeline(steps=[('column_transform',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   MinMaxScaler())]),
                                                  ['intelligence_score',
                                                   'strength_score',
                                                   'speed_score',
                                                   'durability_score',
                                                   'power_score',
                                                   'combat_score']),
                                                 ('words',
                                                  Pipeline(steps=[('tokenizer',
                                                                   CountVectorizer(ngram_range=(1,
                                                                                                2),
                                                                                   tokenizer=<__main__.StemmerTokenizer object at 0x7f286e1ff6d0>)),
                                                                  ('toarray',
                                                                   FunctionTransformer(accept_sparse=True,
                                                                                       func=<function toarray at 0x7f28703dd9e0>))]),
                                                  'history_text')]))])

Se observa en el diagrama de arriba que representa el _pipeline_,
como esta transformación, que se aplica luego de `CountVectorizer`,
garantiza que todos los tipos de columnas pasen al siguiente paso
como objeto `np.array`.


Ahora creamos el _pipeline_ que incluye el paso de selección
de _features_, y finalmente los ajuste de modelos.

In [ ]:
clf_pipe = Pipeline(
    steps=[
        ("preprocessing", preprocessing),
        ("selection", SelectPercentile(f_classif)),
        ("clf", LogisticRegression())
    ]
)

clf_pipe

Pipeline(steps=[('preprocessing',
                 Pipeline(steps=[('column_transform',
                                  ColumnTransformer(transformers=[('num',
                                                                   Pipeline(steps=[('scaler',
                                                                                    MinMaxScaler())]),
                                                                   ['intelligence_score',
                                                                    'strength_score',
                                                                    'speed_score',
                                                                    'durability_score',
                                                                    'power_score',
                                                                    'combat_score']),
                                                                  ('words',
                                                                   Pipeline(steps=[('tokenizer',
                                                                                    CountVectorizer(ngram_range=(1,
                                                                                                                 2),
                                                                                                    tokenizer=<__main__.StemmerTokenizer object at 0x7f286e1ff6d0>)),
                                                                                   ('toarray',
                                                                                    FunctionTransformer(accept_sparse=True,
                                                                                                        func=<function toarray at 0x7f28703dd9e0>))]),
                                                                   'history_text')]))])),
                ('selection', SelectPercentile()),
                ('clf', LogisticRegression())])

Ahora estamos en condiciones para crear nuestra grilla de parámetros,
la que contiene distintos valores sobre las funciones de transformación
como las aplicadas por `CountVectorizer`, pero además, diferentes candidatos de modelos con distintos valores de hiperparámetros. Esto conformará el espacio de búsqueda donde `HalvingGridSearchCV` buscará el mejor modelo según alguna métrica, y en cada iteración, solo pasara la cantidad de candidatos con 
mejor desempeño para reducir el número de candidatos en la próximo iteración 
en un `factor=10` al indicado.


In [ ]:
from lightgbm import LGBMClassifier
rnd_state=666

# Creamos una lista de diccionario con las grillas de parámetro a explorar

param_grid = [
      # Clasificador 1: LogisticRegression
      {
      'preprocessing__column_transform__words__tokenizer__ngram_range': [(1,1), (1,2), (1,3)],
      'preprocessing__column_transform__words__tokenizer__max_df': [0.6, 0.75],
      'preprocessing__column_transform__words__tokenizer__max_features': [25000, 35000],
      'selection__percentile': [20, 40, 60, 80],
      'clf': [LogisticRegression(random_state=rnd_state)],
      'clf__C': [5.3, 5.5],
      'clf__max_iter': [3000],
      'clf__penalty': ['l1'],
      'clf__solver': ['liblinear'],
      'clf__class_weight': ['balanced']
    },
    # Clasificador 2: SupperVectorMachine
    {
     'preprocessing__column_transform__words__tokenizer__ngram_range': [(1,1), (1,2), (1,3)],
     'preprocessing__column_transform__words__tokenizer__max_df': [0.5, 0.75],
     'preprocessing__column_transform__words__tokenizer__max_features': [25000, 35000],
     'selection__percentile': [20, 40, 60, 80],
     'clf': [SVC(decision_function_shape='ovo', random_state=rnd_state)],
     'clf__kernel': ['linear', 'rbf'],
     'clf__C': [1/4, 1/6],
     'clf__gamma': [1.0, 1.1]
    },
    # Clasificador 3: LGBMClasssifier
    {
     'preprocessing__column_transform__words__tokenizer__ngram_range': [(1,1), (1,2), (1,3)],
     'preprocessing__column_transform__words__tokenizer__max_df': [0.5, 0.75],
     'preprocessing__column_transform__words__tokenizer__max_features': [25000, 35000],
     'selection__percentile': [20, 40, 60, 80],
     'clf': [LGBMClassifier(objective='multiclass', random_state=rnd_state, n_jobs=-1)],
     'clf__num_classes': [3],
     'clf__alpha': [0.1],
     'clf__learning_rate': [0.01, 0.1]
    },
]

In [ ]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

search = (HalvingGridSearchCV(clf_pipe, param_grid, n_jobs=-1, 
                              scoring='f1_macro', cv=3, verbose=10,
                              refit=True, aggressive_elimination=True,
                              factor=10)
                                  .fit(X_train, y_train))

- Debido al alto tiempo de ejecución de la grilla generada (aproximadamente 1 horas y media), se serializa la grilla obtenida mediante el siguiente comando:
```python
pickle.dump(search, open("grid_search.p", "wb"))
```

- El archivo serializado está disponible en github en el repositorio del grupo:
`https://github.com/alcazar90/sci-prog-lab/raw/main/laboratorios/grid_search.p`

In [ ]:
# Se descargar el archivo serializado directamente desde el repo
!wget https://github.com/alcazar90/sci-prog-lab/raw/main/laboratorios/grid_search.p

# Se carga archivo serializado
search = pickle.load(open("grid_search.p", "rb"))

--2022-11-10 20:00:09--  https://github.com/alcazar90/sci-prog-lab/raw/main/laboratorios/grid_search.p
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/alcazar90/sci-prog-lab/main/laboratorios/grid_search.p [following]
--2022-11-10 20:00:09--  https://raw.githubusercontent.com/alcazar90/sci-prog-lab/main/laboratorios/grid_search.p
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17363962 (17M) [application/octet-stream]
Saving to: ‘grid_search.p’

grid_search.p       100%[===================>]  16.56M  --.-KB/s    in 0.1s    

2022-11-10 20:00:10 (137 MB/s) - ‘grid_search.p’ saved [17363962/173639

Obtener el puntaje de la mejor configuración de hiperparámetros (`f1_macro`):

In [ ]:
search.best_score_

0.3579534558437512

Revisemos cual fue la mejor configuración de parámetros.

In [ ]:
# Obtener la combinación de parámetros del pipeline con mejor puntaje
search.best_params_

{'clf': LGBMClassifier(alpha=0.1, num_classes=3, objective='multiclass',
                random_state=666),
 'clf__alpha': 0.1,
 'clf__learning_rate': 0.1,
 'clf__num_classes': 3,
 'preprocessing__column_transform__words__tokenizer__max_df': 0.75,
 'preprocessing__column_transform__words__tokenizer__max_features': 35000,
 'preprocessing__column_transform__words__tokenizer__ngram_range': (1, 3),
 'selection__percentile': 80}

Cuales fueron los top 3 configuraciones de modelos:

In [ ]:
# Obtener un subconjunto de columnas del reporte completo de la búsqueda
# de hiperparámetros...
top3_reporte = (pd.DataFrame(search.cv_results_)
        [['params', 'mean_test_score', 'std_test_score', 'rank_test_score']]
        .sort_values('rank_test_score')
        .head(n=3)
        .style
        .set_properties(subset=['params'], **{'width': '650px'})
)

top3_reporte

,params,mean_test_score,std_test_score,rank_test_score
426,"{'clf': LGBMClassifier(alpha=0.1, num_classes=3, objective='multiclass', random_state=666), 'clf__alpha': 0.1, 'clf__learning_rate': 0.1, 'clf__num_classes': 3, 'preprocessing__column_transform__words__tokenizer__max_df': 0.75, 'preprocessing__column_transform__words__tokenizer__max_features': 35000, 'preprocessing__column_transform__words__tokenizer__ngram_range': (1, 3), 'selection__percentile': 80}",0.357953,0.058790,1
30,"{'clf': LogisticRegression(random_state=666), 'clf__C': 5.3, 'clf__class_weight': 'balanced', 'clf__max_iter': 3000, 'clf__penalty': 'l1', 'clf__solver': 'liblinear', 'preprocessing__column_transform__words__tokenizer__max_df': 0.75, 'preprocessing__column_transform__words__tokenizer__max_features': 25000, 'preprocessing__column_transform__words__tokenizer__ngram_range': (1, 2), 'selection__percentile': 60}",0.328571,0.111117,2
51,"{'clf': LogisticRegression(random_state=666), 'clf__C': 5.5, 'clf__class_weight': 'balanced', 'clf__max_iter': 3000, 'clf__penalty': 'l1', 'clf__solver': 'liblinear', 'preprocessing__column_transform__words__tokenizer__max_df': 0.6, 'preprocessing__column_transform__words__tokenizer__max_features': 25000, 'preprocessing__column_transform__words__tokenizer__ngram_range': (1, 1), 'selection__percentile': 80}",0.328571,0.111117,2


Analicemos el reporte de clasificación para la mejor configuración:

In [ ]:
# Métricas del mejor modelo
print(classification_report(y_test, search.predict(X_test)))

              precision    recall  f1-score   support

         Bad       0.57      0.60      0.59       142
        Good       0.73      0.82      0.78       246
     Neutral       0.50      0.03      0.05        37

    accuracy                           0.68       425
   macro avg       0.60      0.48      0.47       425
weighted avg       0.66      0.68      0.65       425



La cantidad de columnas seleccionados.

In [ ]:
# Columnas seleccionadas
cols_selec_modelo_pos = pd.Series(
    search.best_estimator_[1].get_support()
).replace(
    [True, False], ["Columna seleccionada", "Columna no seleccionada"]
)

cols_selec_modelo_pos.value_counts()

Columna seleccionada       28004
Columna no seleccionada     7002
dtype: int64

```
Escriba aquí su justificación
```

**Reporte la mejor combinación encontrada**

La mejor combinación encontrada corresponde al modelo **LGBMClassifier** con
los hiperparámetros reportados a continuación.
```
{'clf': LGBMClassifier(alpha=0.1, num_classes=3, objective='multiclass',
                random_state=666),
 'clf__alpha': 0.1,
 'clf__learning_rate': 0.1,
 'clf__num_classes': 3,
 'preprocessing__column_transform__words__tokenizer__max_df': 0.75,
 'preprocessing__column_transform__words__tokenizer__max_features': 35000,
 'preprocessing__column_transform__words__tokenizer__ngram_range': (1, 3),
 'selection__percentile': 80}
 ```

En cuanto al procesamiento de texto, se selecciona el uso de 1 a 3 "n-gramas", considerando las 35.000 palabras más frecuentes, ignorando aquellas palabras que estén presentes en 75% o más de los documentos ingresados.

Sobre la selección de atributos, se escoge el valor máximo explorado, de percentil 80. Esto quiere decir que existe la posibilidad de que se seleccionara aún más atributos.

**Justifique por qué cree que es la mejor según el clasificador usado**

La selección mediante búsqueda de grilla se hace considerando todas las combinaciones entregadas para todos los hiperparámetros, y además haciendo un proceso de validación cruzada subdividiendo los datos de entrenamiento en entrenamiento y validación en 3 iteraciones. 

Dado lo anterior, se garantiza robustez en los resultados obtenidos mediante la búsqueda de grilla, que hace la selección de atributos según el major valor de métrica de desempeño obtenido promedio (entre las 3 ejecuciones de validación cruzada) para una combinación determinada de hiperparámetros.

El método de búsqueda utiizado es `HalvingGridSearchCV` que comienza
explorando todos los posibles candidatos (aka combinaciones) pero con un
bajo uso de "recursos". Esto significa que se ajustan los modelos solo sobre un
subconjunto del total de las observaciones para entrenar. Luego, y que
explica la palabra _halving_ en su nombre, reduce el grupo de posibles
candidatos para continuar la búsquedaa según lo indicado por el parámetros
`factor`. Utilizamos un factor de 10 debido al gran tamaño del espacio
de búsqueda de hiperparámetros que especificamos en la grila. De esta manera,
por ejemplo, si fueran 100 posibles configuraciones, en la siguiente iterración luego de una selección basada en un _ranking_ solo pasarían 10 candidatos
con un `factor=10`.

Creemos que la selección del modelo `LGBMC`, cuya configuración
presentada arriba, se debe a la gran expresividad de esta familia de modelos.
Recordemos que este tipo de modelos son basados en ensamblajes de muchas
instancias de arboles de clasificación. Cada una de estas instancias tuvieron
la oportunidad de ver un subconjunto de _features_ diferentes, y no la misma
cantidad, cosa que resulta ser de gran utiidad cuando hay muchas
caracteristicas como es el caso, ya que permite identificar cuales son las más relevantes sin que necesariamente implique un sobreajuste o no utilizarla. En resumen, es un modelo con alta expresividad, y que puede insumir gran número de variables con a capacidad de controlar el sobreajuste sin sacrificar información.
 
Adicionalmente, nos llama la atención que el siguiente modelo seleccionado sea una regresión logística (ver reporte top 3 arriba). Sin embargo, el buen
desempeño (luego de `LGMC`), se puede deber al fuerte componente
de regularizaación utilizado `C=5.3` y la penalización de norma 1 que actua
como un seleccionador de variables implicitas. Lo que debe haber ayudado 
a la regresión logistica a lidiar con un problema donde hay más
_features_ que observaciones.

Se debe tener en cuenta además el problema de las clases desbalanceadas. Creemos que realizando una técnica de balanceo de clases el desempeño del modelo mejoraría. Probablemente también realizando un pre procesamiento manual del texto, eliminando palabras más frecuentes presentes en todas las categorías, eliminando puntuaciones, aplicando lemmatización en lugar de stemmización, etc.

**Cantidad de columnas seleccionadas**

El procesamiento del pipeline de procesamiento genera un total de 35.006 columnas (35.000 palabras más frecuentes, más 6 atributos numéricos). Dado que el mejor valor de percentil de selección corresponde a 80%, el número de columnas seleccionadas corresponde a 28.004 columnas.

**Parámetros de CountVectorizer seleccionados por GridSearch**

Como se mencionó y profundizó su justificación previamente, los parámetros de CountVectorizer seleccionados corresponden a las 35.000 palabras más frecuentes, con n gramas de 1 a 3 palabras, y omitiendo las palabras con presencia en 75% o más de los documentos.

## 1.4 Predicción del datos sin etiquetado  [0.5 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/DolotxUUYAAbg7f.jpg" width="350">
</p>


LLego el momento de predecir 
`Vergil`, `Gorilla Girl` y `Batcow`


**Nota:** Recuerde que pueden existir campos vacios en `history_text`, por lo que se les recomienda borrar los nan.

**Respuesta:**

Seleccionamos los atributos relevantes del conjunto de datos y
eliminamos observaciones con posibles valores nan en la columna `history_text`.

In [ ]:
X_no_label = (df_comics_no_label
                .drop_duplicates(subset=['name', 'real_name'])
                [atributos_de_interes + [atributo_a_tokenizar]]
                .loc[~df_comics_no_label[atributo_a_tokenizar].isna(), :]
)

X_no_label.head(n=3)

,intelligence_score,strength_score,speed_score,durability_score,power_score,combat_score,history_text
0,100,20,30,50,35,100,He was one of the many prisoners of Indian Hil...
2,85,30,70,55,65,100,"As with most of the naaru, little is known of ..."
3,90,10,25,25,35,80,"During mid-late 1973, Zero was a member of Tea..."


Se obtienen las predicciones para todas observaciones en `X_no_label`,
se utiliza para esto el _pipeline_ con el mejor modelo ajustado según el
método `HavingGridSearchCV` y la métrica `f1_macro`.

In [ ]:
filter_names = ['Vergil', 'Gorilla Girl', 'Batcow']

(df_comics_no_label
  .drop_duplicates(subset=['name', 'real_name'])
  .loc[~df_comics_no_label[atributo_a_tokenizar].isna(), :]
  [['name', 'real_name']]
  .assign(alignement=search.predict(X_no_label))
  .loc[df_comics_no_label.name.isin(filter_names)]
)

,name,real_name,alignement
16,Batcow,None,Good
40,Gorilla Girl,Fahnbullah Eddy,Good
78,Vergil,Vergil Sparda,Good


Nuestro mejor modelo predice que todo los personajes solicitados son heroes,
o al menos, se encuentran alineados del lado "bueno".

A continuación se analizan las historias de cada uno de los tres personajes:

In [ ]:
pd.set_option('display.max_colwidth', None)

(df_comics_no_label
 .drop_duplicates(subset=['name', 'real_name'])
 .loc[df_comics_no_label.name.isin(filter_names), 'history_text']
)

16    Bat-Cow was originally a cow that was found by Robin during an investigation on a slaughterhouse, which the cow was sent to. Robin wanted to keep the cow and named it Bat-Cow. Batman decided they would keep the cow. During a later investigation, Batman discovered it had mind-controlling radiation inside of it, leading him to attempt to get rid of the cow. Robin, however would not allow him to hurt the cow. After the events of Robin's death, Bat-Cow and Robin's dog felt a great loss in morale. Alfred Pennyworth and Nightwing had to take care of the cow. Later after Nightwing's disappearance, Alfred was the only caretaker of Bat-Cow. Bat-Cow, complete with cape, spent time on a Wayne Enterprises-owned dairy farm. It was there the cow encountered the cosmic-powered "Forever People." One of them seemed to gain guidance from Bat-Cow's actions, namely when Bat-Cow gave one of the Forever People a scarecrow's hat. Bat-Cow formed a friendship with Titus, a dog and household cat.
40      

Algunos extractos interesantes de las historias de personajes que
búscamos predecir su _"aligment"_:

* ✅ - ❌ **Bat-Cow**: podría ser neutro, sin embargo, se encontró siempre rodeado
por súperherores luego de que fuera rescatada por estos. Se muestra que la
"super-mascota" tiene capacidad de generar lazos de amistad. Díficil
que sea un villano, pero podría ser un personaje "Neutro". 
> _"One of them seemed to gain guidance from Bat-Cow's actions, namely when Bat-Cow gave one of the Forever People a scarecrow's hat. Bat-Cow formed a friendship with Titus, a dog and household cat."_
* ✅ **Gorilla Girl**: Definitivamente un personaje bueno, el cual en su momento
fue manipulado por _Moondark_, pero no actuaba bajo su voluntad. Hay registros
de acciones heroicas posteriores la manipulación.
> _"Fahnbulah Eddy (aka Gorilla girl)...afther the incident with Moondark, Eddy and her fellow carnival performers Six and Muck briefly operated as a superhearo team called the Freaks. In their only known outing as such, the team save a doctor from being kidnapped"_
* ❌ **Vergil**: Personaje intrincado y de naturaleza dual. ¿Implicar ser malo cuando se tiene un rol antagonista? La historia lo posiciona como antagonista de su hermano gemelo Dante, pero no aparece ver registros en esta historia de acciones "malvadas" que lo puedan catalogar como villano. 
> _"acting as the main antagonist of Devil May Crey...Despite being the antagonist, Vergill rarely acted beside Dante"_

Al parecer, el único personaje que se identificó incorrectamente fue
**Vergil**. Sin embargo, en la hitoria de este personaje, si bien es
antagonista y uno podría catalogarlo de malo, no aparece en su
historia registros con intenciones directas de realizar daño a su contraparte
Dante. Por lo que quizás, la predicción no se encuentren del todo mal, 
dado los antecedentes disponibles.


<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>